In [114]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Platform (Unified) client library: Custom training image classification model for batch prediction with unmanaged dataset

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/showcase_custom_image_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/showcase_custom_image_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use the AI Platform (Unified) Python client library to train and deploy a custom image classification model for batch prediction.

### Dataset

The dataset used for this tutorial is the [cifar10 dataset](https://www.tensorflow.org/datasets/catalog/cifar10) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). The version of the dataset you will use is built into TensorFlow. The trained model predicts which type of class an image is from ten classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck.

### Objective

In this notebook, you create a custom model from a Python script in a Docker container using the AI Platform (Unified) client library, and then do a prediction on the deployed model by sending data. You can alternatively create custom models using `gcloud` command-line tool or online using the Cloud Console.

The steps performed include:

- Create an AI Platform (Unified) custom job for training a model.
- Train a TensorFlow model.
- Retrieve and load the model artifacts.
- View the model evaluation.
- Upload the model as a AI Platform (Unified) `Model` resource.
- Deploy the `Model` resource to a serving `Endpoint` resource.
- Make a prediction.
- Undeploy the `Model` resource.

### Costs

This tutorial uses billable components of Google Cloud (GCP):

* AI Platform (Unified)
* Cloud Storage

Learn about [Cloud AI Platform
pricing](https://cloud.google.com/ai-platform-unified/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the latest (preview) version of AI Platform (Unified) client library.

In [115]:
! pip3 install -U git+https://github.com/googleapis/python-aiplatform.git@mb-release

  Cloning https://github.com/googleapis/python-aiplatform.git (to revision mb-release) to /private/var/folders/0d/kr5xxsl171x_0fqry12b_3hh00rmmq/T/pip-req-build-lnf178lc
  Running command git clone -q https://github.com/googleapis/python-aiplatform.git /private/var/folders/0d/kr5xxsl171x_0fqry12b_3hh00rmmq/T/pip-req-build-lnf178lc
^C
ERROR: Operation cancelled by user


Install the latest GA version of *google-cloud-storage* library as well.

In [72]:
! pip3 install -U google-cloud-storage

Install the *pillow* library for loading images.

In [73]:
! pip3 install -U pillow

Install the *numpy* library for manipulation of image data.

In [74]:
! pip3 install -U numpy

### Restart the kernel

Once you've installed everything, you need to restart the notebook kernel so it can find the packages.

In [75]:
import os
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Select a GPU runtime

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select "Runtime --> Change runtime type > GPU"**

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the AI Platform (Unified) API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

4. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

5. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [8]:
PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

env: IS_TESTING='1'


In [4]:
Otherwise, set your project ID here.

Project ID:  python-docs-samples-tests


if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "python-docs-samples-tests"  # @param {type:"string"}

In [5]:
#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [6]:
### Authenticate your Google Cloud account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "AI Platform"
into the filter box, and select
   **AI Platform Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [7]:
### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. AI Platform runs
the code from this package. In this tutorial, AI Platform also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then
create AI Platform model and endpoint resources in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where AI Platform (Unified) services are
available](https://cloud.google.com/ai-platform-unified/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with AI Platform.

env: IS_TESTING='1'


BUCKET_NAME = "gs://[your-bucket-name]" #@param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [10]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

In [11]:
**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

! gsutil mb -l $REGION $BUCKET_NAME

In [12]:
Finally, validate access to your Cloud Storage bucket by examining its contents:

Creating gs://python-docs-samples-testsaip-20210414145309/...


! gsutil ls -al $BUCKET_NAME

In [13]:
### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

#### Import AI Platform (Unified) client library

Import the AI Platform (Unified) client library into your Python environment and initialize it.

import os
import sys

from google.cloud.aiplatform import gapic as aip
from google.cloud import aiplatform

aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_NAME)

In [16]:
#### Set hardware accelerators

You can set hardware accelerators for both training and prediction.

Set the variables `TRAIN_GPU/TRAIN_NGPU` and `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each VM, you would specify:

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

For available accelerators, please see: https://cloud.google.com/ai-platform-unified/docs/general/locations#accelerators

Otherwise specify `(None, None)` to use a container image to run on a CPU.

*Note*: TF releases before 2.3 for GPU support will fail to load the custom model in this tutorial. It is a known issue and fixed in TF 2.3 -- which is caused by static graph ops that are generated in the serving function. If you encounter this issue on your own custom models, use a container image for TF 2.3 with GPU support.

TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

DEPLOY_GPU, DEPLOY_NGPU = (None, None)

In [29]:
#### Set pre-built containers

AI Platform (Unified) provides pre-built containers to run training and prediction.

For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers) and [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)

TRAIN_VERSION = 'tf-gpu.2-3'
DEPLOY_VERSION = 'tf2-cpu.2-3'

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

In [97]:
#### Machine Type

Next, set the machine type to use for training and prediction.

- Set the variables `TRAIN_COMPUTE` and `DEPLOY_COMPUTE` to configure  the compute resources for the VMs you will use for for training and prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: The following is not supported for training:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

Training: gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest AcceleratorType.NVIDIA_TESLA_K80 1
Deployment: gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest None None


MACHINE_TYPE = 'n1-standard'

VCPU = '4'
TRAIN_COMPUTE = MACHINE_TYPE + '-' + VCPU
print('Train machine type', TRAIN_COMPUTE)

MACHINE_TYPE = 'n1-standard'

VCPU = '4'
DEPLOY_COMPUTE = MACHINE_TYPE + '-' + VCPU
print('Deploy machine type', DEPLOY_COMPUTE)

In [98]:
# Tutorial

Now you are ready to start creating your own custom model and training for CIFAR10.

Train machine type n1-standard-4
Deploy machine type n1-standard-4


## Train a model

There are two ways you can train a custom model using a container image:

- **Use a Google Cloud prebuilt container**. If you use a prebuilt container, you will additionally specify a Python package to install into the container image. This Python package contains your code for training a custom model.

- **Use your own custom container image**. If you use your own container, the container needs to contain your code for training a custom model.

### Define the command args for the training script

Prepare the command-line arguments to pass to your training script.
- `args`: The command line arguments to pass to the corresponding Python module. In this example, they will be:
  - `"--epochs=" + EPOCHS`: The number of epochs for training.
  - `"--steps=" + STEPS`: The number of steps (batches) per epoch.
  - `"--distribute=" + TRAIN_STRATEGY"` : The training distribution strategy to use for single or distributed training.
     - `"single"`: single device.
     - `"mirror"`: all GPU devices on a single compute instance.
     - `"multi"`: all GPU devices on all compute instances.

JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = '{}/{}'.format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
    "--distribute=" + TRAIN_STRATEGY
]

In [99]:
#### Task.py contents

In the next cell, you will write the contents of the training script task.py. There's no need to go into detail, it's just there for you to browse. In summary:

- Get the directory where to save the model artifacts from the environment variable `AIP_MODEL_DIR`. This variable is set by the training service.
- Loads CIFAR10 dataset from TF Datasets (tfds).
- Builds a model using TF.Keras model API.
- Compiles the model (`compile()`).
- Sets a training distribution strategy according to the argument `args.distribute`.
- Trains the model (`fit()`) with epochs and steps according to the arguments `args.epochs` and `args.steps`.
- Applies a serving function to the trained model to handle base64-encoded images.
- Saves the trained model (`save(MODEL_DIR)`) to the specified model directory.

#### Serving function for image data

To pass images to the prediction service, you encode the bytes into base 64, a convenient way to represent binary data as a string. You need to ensure that the base 64 encoded data gets converted back to raw bytes before it is passed as input to the deployed model.

To resolve this, define a serving function (`serving_fn`) and attach it to the model as a preprocessing step. Add a `@tf.function` decorator so the serving function is part of the model's graph (instead of upstream on a CPU).

When you send a prediction or explanation request, the content of the request is base 64 decoded into a Tensorflow string, which is passed to the serving function (`serving_fn`). The serving function preprocesses the tf.string into raw numpy bytes (`preprocess_fn`) to match the input requirements of the model:
- `io.decode_jpeg`- Decompresses the JPG image which is returned as a Tensorflow vector with three channels (RGB).
- `image.convert_image_dtype` - Changes integer pixel values to float 32 and normalizes the values between the range 0 and 1.
- `image.resize` - Resizes the image to match the input shape for the model.
- `resized / 255.0` - Rescales (normalization) the pixel data between 0 and 1.

At this point, the data can be passed to the model (`m_call`).

In [ ]:
%%writefile task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def make_datasets_unbatched():
  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model

# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
MODEL_DIR = os.getenv("AIP_MODEL_DIR")

train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(MODEL_DIR)

# CONCRETE_INPUT = "numpy_inputs"

# # Decode jpeg data, rescale and convert to floats
# def _preprocess(bytes_input):
#     decoded = tf.io.decode_jpeg(bytes_input, channels=3)
#     decoded = tf.image.convert_image_dtype(decoded, tf.float32)
#     resized = tf.image.resize(decoded, size=(32, 32))
#     rescale = tf.cast(resized / 255.0, tf.float32)
#     return rescale


# @tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
# def preprocess_fn(bytes_inputs):
#     decoded_images = tf.map_fn(_preprocess, bytes_inputs, dtype=tf.float32, back_prop=False)
#     return {CONCRETE_INPUT: decoded_images}  # User needs to make sure the key matches model's input


# m_call = tf.function(model.call).get_concrete_function([tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)])


# @tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
# def serving_fn(bytes_inputs):
#     images = preprocess_fn(bytes_inputs)
#     prob = m_call(**images)
#     return prob


# tf.saved_model.save(model, MODEL_DIR, signatures={
#     'serving_default': serving_fn,
# })

In [100]:
### Train the model


Now start the training of your custom training job on AI Platform (Unified).

Use the CustomTrainingJob class to define the job, which takes the following parameters:

- `display_name`: The user-defined name of this training pipeline.
- `script_path`: The local path to the training script.
- `container_uri`: The Uri of the training container image.
- `requirements`: The list of python packages dependencies of the script.
- `model_serving_container_image_uri`: If the training produces a managed AI Platform Model, the URI of the Model serving container suitable for serving the model produced by the training script.

Use the `run` function on the training job to start training, which takes the following parameters:

- `args`: The command line arguments to be passed to the Python script.
- `replica_count`: The number of worker replicas.
- `model_display_name`: The display name of the `Model` if the script produces a managed `Model`.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.

This will create a training pipeline that trains and creates a `Model` object. This is then returned by the function.

Overwriting task.py


DEPLOY_IMAGE

In [101]:
job = aiplatform.CustomTrainingJob(display_name=JOB_NAME,
                                   script_path='task.py',
                                   container_uri=TRAIN_IMAGE,
                                   requirements=['tensorflow_datasets==1.3.0'],
                                   model_serving_container_image_uri=DEPLOY_IMAGE)

MODEL_DISPLAY_NAME = "cifar10-" + TIMESTAMP

# Start the training
if TRAIN_GPU:
    model = job.run(model_display_name=MODEL_DISPLAY_NAME,
                    args=CMDARGS,
                    replica_count=1,
                    machine_type=TRAIN_COMPUTE,
                    accelerator_type=TRAIN_GPU.name,
                    accelerator_count=TRAIN_NGPU
                   )
else:
    model = job.run(model_display_name=MODEL_DISPLAY_NAME,
                    args=CMDARGS,
                    replica_count=1,
                    machine_type=TRAIN_COMPUTE,
                    accelerator_count=0
                   )

'gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest'

In [102]:
# For base64 encoded image
# model = aiplatform.Model("1861059769452724224")

INFO:google.cloud.aiplatform.training_jobs:Training script copied to:
gs://python-docs-samples-testsaip-20210414145309/aiplatform-2021-04-14-17:03:57.404-aiplatform_custom_trainer_script-0.1.tar.gz.
INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://python-docs-samples-testsaip-20210414145309/aiplatform-custom-training-2021-04-14-17:03:57.771 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3420035316243759104?project=1012616486416
INFO:google.cloud.aiplatform.training_jobs:Training projects/1012616486416/locations/us-central1/trainingPipelines/3420035316243759104 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:Training projects/1012616486416/locations/us-central1/trainingPipelines/3420035316243759104 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:Training projects/1012616486416/locat

In [17]:
### Compute instance scaling

You have several choices on scaling the compute instances for handling your batch prediction requests:

- Single Instance: The online prediction requests are processed on a single compute instance.
  - Set the minimum (`MIN_NODES`) and maximum (`MAX_NODES`) number of compute instances to one.


- Manual Scaling: The online prediction requests are split across a fixed number of compute instances that you manually specified.
  - Set the minimum (`MIN_NODES`) and maximum (`MAX_NODES`) number of compute instances to the same number of nodes. When a model is first deployed to the instance, the fixed number of compute instances are provisioned and online prediction requests are evenly distributed across them.

The minimum number of compute instances corresponds to the field `min_replica_count` and the maximum number of compute instances corresponds to the field `max_replica_count`, in your subsequent deployment request.

MIN_NODES = 1
MAX_NODES = 1

In [103]:
import tensorflow as tf

loaded = tf.saved_model.load("gs://python-docs-samples-testsaip-20210414145309/aiplatform-custom-training-2021-04-14-17:03:57.771/model")

input_name = list(loaded.signatures['serving_default'].structured_input_signature[1].keys())[0]
print('Serving function input:', input_name)



In [116]:
# Download the images
! gsutil -m cp -r gs://cloud-samples-data/ai-platform-unified/cifar_test_images .

import numpy as np
from PIL import Image
import base64

# Load image data
IMAGE_DIRECTORY = "cifar_test_images"

image_files = [file for file in os.listdir(IMAGE_DIRECTORY) if file.endswith(".jpg")]

# Decode JPEG images into numpy arrays
image_data = [
    np.asarray(Image.open(os.path.join(IMAGE_DIRECTORY, file))) for file in image_files
]

# x_test = [np.asarray(open(os.path.join(IMAGE_DIRECTORY, file), 'rb').read()) for file in image_files]
# x_test_b64 = [{ "b64": base64.b64encode(x).decode('utf-8') } for x in x_test]

# Scale and convert to expected format
x_test = [(image / 255.0).astype(np.float32).tolist() for image in image_data]

# Extract labels from image name
y_test = [int(file.split("_")[1]) for file in image_files]


Serving function input: conv2d_input


In [50]:
model.resource_name

'projects/1012616486416/locations/us-central1/models/1861059769452724224'

In [117]:
%env IS_TESTING '1'

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_0_3.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_0_6.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_1.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_5.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_8.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_9.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_3_7.jpg...
Copying gs://cloud-samples-data/ai-platf

In [118]:
import json

BATCH_PREDICTION_INSTANCES_FILE = 'batch_prediction_instances.jsonl'

# Upload to GCS bucket
BATCH_PREDICTION_GCS_SOURCE = BUCKET_NAME + "/batch_prediction_instances/" + BATCH_PREDICTION_INSTANCES_FILE

with open(BATCH_PREDICTION_INSTANCES_FILE, 'w') as f:
    for x in x_test:
        data = {input_name: x}
        f.write(json.dumps(data) + '\n')

! gsutil cp $BATCH_PREDICTION_INSTANCES_FILE $BATCH_PREDICTION_GCS_SOURCE

BATCH_PREDICTION_GCS_SOURCE

Copying file://batch_prediction_instances.jsonl [Content-Type=application/octet-stream]...

Operation completed over 1 objects/617.5 KiB.                                    


'gs://python-docs-samples-testsaip-20210414145309/batch_prediction_instances/batch_prediction_instances.jsonl'

In [57]:
# BATCH_PREDICTION_INSTANCES_FILE = 'batch_prediction_instances.txt'

# ! gsutil -m ls gs://cloud-samples-data/ai-platform-unified/cifar_test_images > $BATCH_PREDICTION_INSTANCES_FILE

# BATCH_PREDICTION_GCS_SOURCE = BUCKET_NAME + "/batch_prediction_instances/" + BATCH_PREDICTION_INSTANCES_FILE

# ! gsutil cp $BATCH_PREDICTION_INSTANCES_FILE $BATCH_PREDICTION_GCS_SOURCE

gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_0_3.jpg
gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_0_6.jpg
gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_1.jpg
gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_5.jpg
gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_8.jpg
gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_9.jpg
gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_3_7.jpg
gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_4_10.jpg
gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_5_4.jpg
gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_7_2.jpg


### Make batch prediction request

Now that your batch of two image test items is ready, let's do the batch request. Use this helper function `create_batch_prediction_job`, with the parameters:

- `display_name`: The human readable name for the prediction job.
- `model_name`: The AI Platform (Unified) fully qualified identifier for the model.
- `gcs_source_uri`: The Cloud Storage path to the JSONL/CSV input file -- which you created above.
- `gcs_destination_output_uri_prefix`: The Cloud Storage path that the service will write the predictions to.
- `parameters`: Additional filtering parameters for serving prediction results.

The helper function uses the job client service and calls the method `create_batch_prediction_job`, with the parameters:

- `parent`: The AI Platform (Unified) location root path for dataset, model and pipeline resources.
- `batch_prediction_job`: The specification for the batch prediction job.

Let's now dive into the specification for the `batch_prediction_job`:

- `display_name`: The human readable name for the prediction batch job.
- `model`: The AI Platform (Unified) fully qualified identifier for the model.
- `model_parameters`: Additional filtering parameters for serving prediction results..
  - `confidence_threshold`: The threshold for returning predictions. Must be between 0 and 1.
  - `max_predictions`: The maximum number of predictions to return per classification, sorted by confidence.
- `input_config`: The input source and format type for the instances to predict.
 - `instances_format`: The format of the batch prediction request file: `csv` or `jsonl`.
 - `gcs_source`: A list of one or more Cloud Storage paths to your batch prediction requests.
- `output_config`: The output destination and format for the predictions.
 - `prediction_format`: The format of the batch prediction response file: `csv` or `jsonl`.
 - `gcs_destination`: The output destination for the predictions.
- `dedicated_resources`: The compute resources to provision for the batch prediction job.
  - `machine_spec`: The compute instance to provision. Use the variable `DEPLOY_GPU != None` to use a GPU; otherwise only a CPU is allocated.
  - `starting_replica_count`: The number of compute instances to initially provision.
  - `max_replica_count`: The maximum number of compute instances to scale to. In this tutorial, only one instance is provisioned.

This call is an asychronous operation. You will print from the response object a few select fields, including:

- `name`: The AI Platform (Unified) fully qualified identifier assigned to the batch prediction job.
- `display_name`: The human readable name for the prediction batch job.
- `model`: The AI Platform (Unified) fully qualified identifier for the model.
- `generate_explanations`: Whether True/False explanations were provided with the predictions (explainability).
- `state`: The state of the prediction job (pending, running, etc).

Since this call will take a few moments to execute, you will likely get `JobState.JOB_STATE_PENDING` for `state`.

The helper function will return and save the AI Platform (Unified) fully qualified identifier assigned to the batch prediction job as `prediction_name`.

In [120]:
from google.cloud.aiplatform import jobs

BATCH_MODEL = "cifar10_batch-" + TIMESTAMP
BATCH_PREDICTION_GCS_DEST_PREFIX = BUCKET_NAME + "/batch_prediction_results"

# Make SDK batch_predict method call
batch_prediction_job = model.batch_predict(
    instances_format="jsonl",
    predictions_format="jsonl",
    job_display_name=BATCH_MODEL,
    gcs_source=BATCH_PREDICTION_GCS_SOURCE,
    gcs_destination_prefix=BATCH_PREDICTION_GCS_DEST_PREFIX,
    model_parameters={'confidenceThreshold': 0.5, 'maxPredictions': 2},
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=True,
)

INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/8605930287160885248?project=1012616486416
INFO:google.cloud.aiplatform.jobs: projects/1012616486416/locations/us-central1/batchPredictionJobs/8605930287160885248 current state:
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs: projects/1012616486416/locations/us-central1/batchPredictionJobs/8605930287160885248 current state:
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs: projects/1012616486416/locations/us-central1/batchPredictionJobs/8605930287160885248 current state:
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs: projects/1012616486416/locations/us-central1/batchPredictionJobs/8605930287160885248 current state:
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs: projects/1012616486416/locations/us-central1/batchPredictionJobs/8605930287160885248 current state:
JobState.JOB_STATE_RUNNING
IN

In [121]:
def get_latest_predictions(gcs_out_dir):
    ''' Get the latest prediction subfolder using the timestamp in the subfolder name'''
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split('/')[-2]
        if subfolder.startswith('prediction-'):
            if subfolder > latest:
                latest = folder[:-1]
    return latest

folder = get_latest_predictions(BATCH_PREDICTION_GCS_DEST_PREFIX)

results_files = ! gsutil ls $folder/prediction.results\*

results_files

results = []
for results_file in results_files:
    ! gsutil cp $results_file 'output'
    with open('output', 'r') as file:
        results.extend([json.loads(line) for line in file.readlines()])

results

Copying gs://python-docs-samples-testsaip-20210414145309/batch_prediction_results/prediction-cifar10-20210414145309-2021_04_14T15_13_06_837Z/prediction.results-00000-of-00001...

Operation completed over 1 objects/619.1 KiB.                                    


   [0.572549045085907, 0.3686274588108063, 0.3843137323856354],
     [0.5137255191802979, 0.3333333432674408, 0.3450980484485626],
     [0.5098039507865906, 0.3333333432674408, 0.3529411852359772],
     [0.5372549295425415, 0.3686274588108063, 0.3960784375667572],
     [0.8627451062202454, 0.6941176652908325, 0.729411780834198]],
    [[0.9960784316062927, 0.9960784316062927, 0.9960784316062927],
     [0.9960784316062927, 0.9960784316062927, 0.9960784316062927],
     [0.9764705896377563, 0.9764705896377563, 0.9764705896377563],
     [0.9764705896377563, 0.9764705896377563, 0.9764705896377563],
     [0.5058823823928833, 0.5058823823928833, 0.5058823823928833],
     [0.0, 0.0, 0.0],
     [0.007843137718737125, 0.007843137718737125, 0.007843137718737125],
     [0.007843137718737125, 0.007843137718737125, 0.007843137718737125],
     [0.003921568859368563, 0.003921568859368563, 0.003921568859368563],
     [0.003921568859368563, 0.003921568859368563, 0.003921568859368563],
     [0.00392156885

In [124]:
y_predicted = [np.argmax(result["prediction"]) for result in results]

correct = sum(y_predicted == np.array(y_test))
accuracy = len(y_predicted)
print(
    f"Correct predictions = {correct}, Total predictions = {accuracy}, Accuracy = {correct/accuracy}"
)

Correct predictions = 5, Total predictions = 10, Accuracy = 0.5


In [104]:
DEPLOYED_NAME = "cifar10_deployed-" + TIMESTAMP
TRAFFIC_SPLIT = {"0": 100}

endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=None,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MIN_NODES,
    )

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_0_3.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_0_6.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_1.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_5.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_8.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_2_9.jpg...
Copying gs://cloud-samples-data/ai-platform-unified/cifar_test_images/image_3_7.jpg...
Copying gs://cloud-samples-data/ai-platf

In [105]:
# Download the images
! gsutil -m cp -r gs://cloud-samples-data/ai-platform-unified/cifar_test_images .

import numpy as np
from PIL import Image
import base64

# Load image data
IMAGE_DIRECTORY = "cifar_test_images"

image_files = [file for file in os.listdir(IMAGE_DIRECTORY) if file.endswith(".jpg")]

# Decode JPEG images into numpy arrays
image_data = [
    np.asarray(Image.open(os.path.join(IMAGE_DIRECTORY, file))) for file in image_files
]

# x_test = [np.asarray(open(os.path.join(IMAGE_DIRECTORY, file), 'rb').read()) for file in image_files]
# x_test_b64 = [{ "b64": base64.b64encode(x).decode('utf-8') } for x in x_test]

# Scale and convert to expected format
x_test = [(image / 255.0).astype(np.float32).tolist() for image in image_data]

# Extract labels from image name
y_test = [int(file.split("_")[1]) for file in image_files]

In [106]:
x_test

3921568859368563, 0.003921568859368563],
   [0.003921568859368563, 0.003921568859368563, 0.003921568859368563],
   [0.1882352977991104, 0.05882352963089943, 0.08627451211214066],
   [0.501960813999176, 0.35686275362968445, 0.3843137323856354],
   [0.7098039388656616, 0.5254902243614197, 0.5568627715110779],
   [0.7137255072593689, 0.47843137383461, 0.5176470875740051],
   [0.7803921699523926, 0.5058823823928833, 0.545098066329956],
   [0.7960784435272217, 0.4941176474094391, 0.5333333611488342],
   [0.7607843279838562, 0.4431372582912445, 0.47843137383461],
   [0.8784313797950745, 0.5686274766921997, 0.5921568870544434],
   [0.8705882430076599, 0.5764706134796143, 0.5960784554481506],
   [0.7803921699523926, 0.5215686559677124, 0.529411792755127],
   [0.6352941393852234, 0.40392157435417175, 0.4117647111415863],
   [0.572549045085907, 0.3686274588108063, 0.3843137323856354],
   [0.5137255191802979, 0.3333333432674408, 0.3450980484485626],
   [0.5098039507865906, 0.3333333432674408, 0.3

In [107]:
predictions = endpoint.predict(instances=x_test)

y_predicted = np.argmax(predictions.predictions, axis=1)

correct = sum(y_predicted == np.array(y_test))
accuracy = len(y_predicted)
print(
    f"Correct predictions = {correct}, Total predictions = {accuracy}, Accuracy = {correct/accuracy}"
)

Correct predictions = 5, Total predictions = 10, Accuracy = 0.5


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Training Job
- Model
- Endpoint
- Cloud Storage Bucket

In [ ]:
delete_training_job = True
delete_model = True
delete_endpoint = True

# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete the training job
job.delete()

# Delete the model
model.delete()

# Delete the endpoint
endpoint.delete()

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil -m rm -r $BUCKET_NAME